In [3]:
import numpy as np
import re
import os
import autokeras as ak
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, LayerNormalization
from ReadData import read_data_as_img, read_data_structured, read_data_st
from Preprocessing import ros, smote, adasyn
from Results import report_results_imagedata, make_spider_by_temp, report_results_st, test_results
from keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from keras import Sequential
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, log_loss, fowlkes_mallows_score, cohen_kappa_score, precision_score, recall_score
from datetime import datetime
from contextlib import redirect_stdout
seed = 42
np.random.seed(seed)

In [ ]:
data_dir = "../data"
X_train, y_train = read_data_st(data_dir, "train")
X_val, y_val = read_data_st(data_dir, "val")
X_test, y_test = read_data_st(data_dir, "test")

#X_train = np.concatenate((X_train, X_val))
#y_train = np.concatenate((y_train, y_val))
#X_train, y_train = smote(X_train, y_train)
X_train = X_train.reshape(*X_train.shape[:3], 1)
X_test = X_test.reshape(*X_test.shape[:3], 1)

In [19]:
def widenet():
    model = keras.Sequential()
    
    model.add(Conv2D(filters=32, kernel_size=(2, 2), activation='relu', input_shape=(11,200,1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=32, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=32, kernel_size=(2, 2), activation='relu'))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(units=256, activation='relu'))

    model.add(Dense(1, activation = 'sigmoid'))

    
    # Compile the model
    # SGD
    # RMSprop
    # Adam
    # Adadelta
    # Adagrad
    # Adamax
    # Nadam
    # Ftrl
    model.compile(optimizer="adam", loss=keras.losses.BinaryCrossentropy(), metrics=["accuracy", "AUC"])

    return model

In [20]:
model = widenet()

model.summary()

model.fit(X_train, y_train,
            shuffle=True,
            batch_size=32,
            epochs=1,
            verbose=True,
            validation_split=0.2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 10, 199, 32)       160       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 99, 32)         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 5, 99, 32)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 98, 32)         4128      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 49, 32)         0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 2, 49, 32)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 1, 48, 32)        

In [21]:
y_test_pred = model.predict(X_test)
y_test_pred_round = y_test_pred.round().astype(int)

In [22]:
y_test_pred

array([[0.14920753],
       [0.29627657],
       [0.3582521 ],
       ...,
       [0.11626548],
       [0.00918728],
       [0.01114103]], dtype=float32)

In [27]:
print(f"\tPredicciones clase 0: {sum(y_test_pred_round == 0).sum()}")
print(f"\tPredicciones clase 1: {sum(y_test_pred_round == 1).sum()}")
print(f"\tInstancias clase 0: {np.count_nonzero(y_test == 0)}")
print(f"\tInstancias clase 1: {np.count_nonzero(y_test == 1)}")

print(f"\tF1 score: {f1_score(y_test, y_test_pred_round)}")
print(f"\tBinary crossentropy: {log_loss(y_test, y_test_pred, eps=1e-15)}")
print(f"\tAccuracy score: {accuracy_score(y_test, y_test_pred_round)}")
print(f"\tAUC ROC: {roc_auc_score(y_test, y_test_pred)}")

	Predicciones clase 0: 11262
	Predicciones clase 1: 0
	Instancias clase 0: 10238
	Instancias clase 1: 1024
	F1 score: 0.0
	Binary crossentropy: 0.23091646231716864
	Accuracy score: 0.909074764695436
	AUC ROC: 0.8617254079471088
